In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json

In [2]:
# Get Fierce Health main page
HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
url = "https://www.fiercehealthcare.com"
page = requests.get(url, headers=HEADERS)

# Check request status
status = page.status_code
if status == 403:
    raise ValueError("Exception thrown as url get request returned 403 error. No access")

# Open list of keywords
with open('keywords.json') as file:
     data = json.load(file)

In [3]:
# Parse html code
# We'll save in coverpage the cover page content
fierceHealth = page.content

# Soup creation
soup1 = BeautifulSoup(fierceHealth, 'html5lib') # using html parser to read through site data

In [4]:
# Scrape html from site
## Source 3 - Hippa Journal ##
# News identification for Source 1
coverpage_news = soup1.find_all('div', id='page-wrapper')       # Select right column with latest news
if not coverpage_news:
    # Throws exception if soup cannot find the main 'container' on the site
    # Only returns exception if the length of coverpage news is 0 ... should be 1 and not 0
    raise ValueError("Error in returning html data from main page. Length of array is 0")

# Accesses direct list of srticles from the page wrapper html id
# Using only one find all instead of 15    
articles = coverpage_news[0].contents[1].contents[3].contents[1].contents[1].contents[1].contents[1].contents[3].contents[1].contents[7].contents[1].contents[3].contents[1].contents[1].contents[3].contents[1].contents[3].contents[1].find_all('div', class_= 'col-md-6 col-lg-12 mb-5 mb-lg-4 pl-lg-5 d-md-flex flex-md-column content-list-wrapper')

In [5]:
# Establish the number of articles
num_of_articles = len(articles)
if not num_of_articles:
    # Throws exception if soup cannot find articles in find_all
    # Only returns exception if the length of articles is 0 ... should be more than 0
    raise ValueError("Error in returning html data. Length of array is 0")

In [6]:
Index = []
list_links = []
list_titles = []
Keyword = []
count = 0
added_articles = set()

# for each article in list of scraped articles 
for n in np.arange(0, num_of_articles):
    
    # Getting the link of the article
    link = articles[n].find('a')['href']
    
    # Getting the title
    title = articles[n].get_text()
    title = title.replace('\n', '') # Remove \n
    title = title.replace('\t', '') # Remove \t
    title = " ".join(title.split()) # Remove spaces in front and back of string
    
    found_keyword = False # initialize found_keyword flag
    
# https://www.geeksforgeeks.org/read-json-file-using-python/
# Filter by Keyword inside "data"
    for entry in data:
        if entry['Keyword'] in title:
            Keyword.append(entry['Keyword'])  # Append Found Keyword
            added_articles.add(title)
            found_keyword = True # set found_keyword flag to True
            break
    
    # append 'none' if no keyword was found
    if not found_keyword:
        Keyword.append('none')
    
    # add article information to lists
    list_links.append(url + link)
    list_titles.append(title)
    Index.append(count)
    count += 1 

In [7]:
 # df_show_info
df_show_info = pd.DataFrame(
    {'ind': Index,
     'keyword': Keyword,
     'Article Title': list_titles,
     'Article Link': list_links})

df_show_info

,ind,keyword,Article Title,Article Link
0,0,none,Bill aimed at virtual mental health in Medicar...,https://www.fiercehealthcare.com/regulatory/le...
1,1,none,SponsoredAgitation: An Unspoken Aspect of Alzh...,https://www.fiercehealthcare.com/sponsored/uns...
2,2,none,Regulatory filing sheds new light on Kaiser-Ge...,https://www.fiercehealthcare.com/providers/kai...
3,3,none,SponsoredRevolutionizing Hospital Care through...,https://www.fiercehealthcare.com/sponsored/rev...
4,4,none,Federal appeals court grants stay in ACA preve...,https://www.fiercehealthcare.com/regulatory/fe...
